In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [97]:
df = px.data.iris()
df['petal_area'] = df['petal_length'] * df['petal_width']
df = df.groupby(by='species').max().reset_index().loc[:, ['species','petal_area']].rename(columns={'petal_area':'petal_area_max'}).merge(df, on='species')
df['petal_area_normalized'] = df['petal_area'] * (1/df['petal_area_max']) #bring petal_area between range [0-1] to put it in as opacity
df

,species,petal_area_max,sepal_length,sepal_width,petal_length,petal_width,species_id,petal_area,petal_area_normalized
0,setosa,0.96,5.1,3.5,1.4,0.2,1,0.28,0.291667
1,setosa,0.96,4.9,3.0,1.4,0.2,1,0.28,0.291667
2,setosa,0.96,4.7,3.2,1.3,0.2,1,0.26,0.270833
3,setosa,0.96,4.6,3.1,1.5,0.2,1,0.30,0.312500
4,setosa,0.96,5.0,3.6,1.4,0.2,1,0.28,0.291667
...,...,...,...,...,...,...,...,...,...
145,virginica,15.87,6.7,3.0,5.2,2.3,3,11.96,0.753623
146,virginica,15.87,6.3,2.5,5.0,1.9,3,9.50,0.598614
147,virginica,15.87,6.5,3.0,5.2,2.0,3,10.40,0.655325
148,virginica,15.87,6.2,3.4,5.4,2.3,3,12.42,0.782609


In [18]:
fig_1 = px.scatter(x='sepal_length', y='sepal_width', width=1300, data_frame=df)
fig_1.update_layout(title='Iris Sepal Plain')
fig_1.show()

In [25]:
fig_2 = px.scatter(x='sepal_length', y='sepal_width', width=1300, size='petal_length', data_frame=df)
fig_2.update_layout(title='Iris Sepal Size')
fig_2.show()

In [27]:
fig_3 = px.scatter(x='sepal_length', y='sepal_width', width=1300, size='petal_length', color='species', data_frame=df)
fig_3.update_layout(title='Iris Sepal Hue & Size')
fig_3.show()

In [72]:
fig_4 = px.scatter(x='sepal_width', y='sepal_length', width=1300, size='petal_length', color='species', data_frame=df)
fig_4.update_layout(title='Iris Sepal Hue & Size & Position')
fig_4.show()

In [98]:
fig_5 = px.scatter(x='sepal_length', y='sepal_width', width=1300, size='petal_length', color='species', opacity=df['petal_area_normalized'], data_frame=df)
fig_5.update_layout(title='Iris Sepal Hue & Size & Value')
fig_5.show()

In [71]:
fig_6 = go.Figure()
symbols=['circle','diamond','x']
i = 0
for species in df['species'].unique():
    df_species = df[df['species'] == species]
    
    fig_6.add_trace(
        go.Scatter(
            name=species,
            mode='markers',
            x=df_species['sepal_length'],
            y=df_species['sepal_width'],
            marker=dict(
                color='rgba(135, 206, 250, 0.5)',
                size=10,
                symbol=symbols[i],
                line=dict(
                    color='darkmagenta',
                    width=1
                )
            ),
            showlegend=True
        )
    )
    i += 1

fig_6.update_layout(width=1300)
fig_6.show()